# Importación de librerías  

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn import metrics
import optuna
import mlflow

import warnings
warnings.filterwarnings('ignore')

# Cargar datos

In [2]:
data = pd.read_csv("../data/clean/personas_limpio.csv")
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1


In [3]:
# Dado que las variables binarias quedaron como númericas, se procede a convertirlas de nuevo
def convert_to_categorical(df, columns):
    """
    Convierte las columnas especificadas de un DataFrame a tipo categórico.

    Parámetros:
    df (pd.DataFrame): El DataFrame en el que se realizarán las conversiones.
    columns (list): Lista de nombres de columnas a convertir a tipo categórico.

    Retorna:
    pd.DataFrame: El DataFrame con las columnas especificadas convertidas a tipo categórico.
    """
    for column in columns:
        if column in df.columns:
            df[column] = pd.Categorical(df[column])
        else:
            print(f"Columna '{column}' no encontrada en el DataFrame.")
    return df

categorical_columns = ['hypertension', 'heart_disease','stroke']
data = convert_to_categorical(data, categorical_columns)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   gender             5110 non-null   object  
 1   age                5110 non-null   float64 
 2   hypertension       5110 non-null   category
 3   heart_disease      5110 non-null   category
 4   ever_married       5110 non-null   object  
 5   work_type          5110 non-null   object  
 6   Residence_type     5110 non-null   object  
 7   avg_glucose_level  5110 non-null   float64 
 8   bmi                5110 non-null   float64 
 9   smoking_status     5110 non-null   object  
 10  stroke             5110 non-null   category
dtypes: category(3), float64(3), object(5)
memory usage: 334.8+ KB


El conjunto de datos `personas_limpio`contiene 5.110 registros y 11 columnas resultado de la limpieza de datos aplicada. Las variables categóricas son `gender`, `ever_married`, `work_type`, `resident_type`y `smoking_status`, variables que indican factores importantes a la hora de predecir un ACV. Además, las variables númericas tales `age`, `avg_glucose_level` y `bmi` indican edad y estado de salud de la persona. Y las variables binarias `hypertension` y `heart_disase`, las cuales indican el padecimiento de cierta patologia. Finalmente, la variable a predecir `stroke` indica la presencia de ACV en la persona.

# Procesamiento de los datos

En este apartado se definen las variables predictoras y a predecir, se codifican las variables categoricas aplicando One-Hot Encoding para transformarlas en variables númericas y se aplica reducción de dimensionalidad con PCA. Además, se divide el conjunto de datos en entrenamiento y prueba para la construcción y prueba de los modelos.

In [5]:
data['stroke'].value_counts()

stroke
0    4861
1     249
Name: count, dtype: int64

In [6]:
# Definición de las variables predictoras y a predecir
X = data.drop(columns=['stroke'])  # Eliminar variable a predecir
y = data['stroke']

In [7]:
# Conversión de variables categoricas a numericas con One-Hot Encoding
data_dummies = pd.get_dummies(X,columns=X.select_dtypes(include=['object','category']).columns.to_list())


Dado que la clase a predecir esta desbalanceada, hay mayor proporción de registros de personas aun no han presentado un ACV, se aplicará una técnica de balanceo para distribuir la clase predictora.

In [26]:
def class_balance(X,y,tecnica='SMOTE'):
    """
    Aplica diversas tecnicas de balanceo de clases.

    Parámetros:
    X: variables predictoras.
    y: variable a predecir.
    técnica: método a aplicar de balanceo de clases.

    Retorna:
    X_resampled: variables predictoras resampleadas.
    y_resampled: variable a predecir resampleada.
    """
    if tecnica == 'SMOTE':
        
        # Oversampling con SMOTE
        smote = SMOTE(sampling_strategy='minority')  # Aumenta solo la clase minoritaria
        X_resampled, y_resampled = smote.fit_resample(X, y)

    if tecnica == 'Undersampling':

        # Undersampling con RandomUnderSampler
        undersample = RandomUnderSampler(sampling_strategy=0.5)  # Reduce la clase mayoritaria a una proporción de 0.5
        X_resampled, y_resampled = undersample.fit_resample(X, y)

    if tecnica == 'SMOTE-ENN':

        # Combinación de SMOTE y ENN(Edicion de vecinos)
        smote_enn = SMOTEENN(sampling_strategy='auto')
        X_resampled, y_resampled = smote_enn.fit_resample(X, y)

    return X_resampled, y_resampled 

In [96]:
X_resampled, y_resampled = class_balance(data_dummies, y, 'SMOTE-ENN')

In [97]:
y_resampled.value_counts()

stroke
1    4607
0    3606
Name: count, dtype: int64

In [98]:
# Estandarización de las variables
scaler = StandardScaler()
data_scaled = scaler.fit_transform(X_resampled)

In [ ]:
# Reducción de dimensionalidad PCA
pca = PCA(n_components=0.95)  # Conservar el 95% de la varianza
data_pca = pca.fit_transform(data_scaled)

print(f"Dimensiones antes de PCA: {data_scaled.shape}")
print(f"Dimensiones después de PCA: {data_pca.shape}")

Dimensiones antes de PCA: (8213, 23)
Dimensiones después de PCA: (8213, 18)


In [100]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_pca, y_resampled, test_size=0.33, random_state=40)

# Ajuste hiperparametros

En esta sección se realizara el ajuste de hiperparametros en cada uno de los modelos de clasificación probados, los mejores hiperparametros seran almacenados en MLFlow.

In [101]:
# Modelo básico
clf = XGBClassifier()
# Entrenamiento (fit) y desempeño del modelo XGBClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.9674690956408588, 0.9631132423459978)

In [102]:
# Se usara una funcion para hacer la busqueda de los mejores hiperparametros de manera automática
def objective(trial):
    """
    Retorna: Métrica del modelo de clasificación
    """
    # Hiperparametros a probar 
    param = {
        'verbosity': 0,
        'objective': 'binary:logistic',  # Clasificación binaria
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 8),
        'reg_lambda' : trial.suggest_int('reg_lambda',1, 10),
        'reg_alpha' : trial.suggest_int('reg_alpha',1, 10)
    }

    # Crear y entrenar el modelo
    clf = XGBClassifier(**param)
    clf.fit(X_train, y_train)

    # Predecir sobre el conjunto de prueba
    y_pred = clf.predict(X_test)

    # Calcular metricas
    acc = metrics.accuracy_score(y_pred, y_test.values)
    f1_score = metrics.f1_score(y_pred, y_test.values)

    return f1_score

In [103]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-11-09 15:57:39,021] A new study created in memory with name: no-name-df244b23-e812-4449-8217-2d4ea516493a
[I 2024-11-09 15:57:39,166] Trial 0 finished with value: 0.9637136319058516 and parameters: {'n_estimators': 189, 'max_depth': 4, 'learning_rate': 0.04857723826746932, 'subsample': 0.7485446920851834, 'colsample_bytree': 0.9601797869858306, 'min_child_weight': 7, 'reg_lambda': 5, 'reg_alpha': 2}. Best is trial 0 with value: 0.9637136319058516.
[I 2024-11-09 15:57:39,451] Trial 1 finished with value: 0.962696335078534 and parameters: {'n_estimators': 217, 'max_depth': 10, 'learning_rate': 0.024596419304756895, 'subsample': 0.7443996860757947, 'colsample_bytree': 0.9746064868195567, 'min_child_weight': 4, 'reg_lambda': 10, 'reg_alpha': 6}. Best is trial 0 with value: 0.9637136319058516.
[I 2024-11-09 15:57:39,517] Trial 2 finished with value: 0.9378457533355028 and parameters: {'n_estimators': 178, 'max_depth': 1, 'learning_rate': 0.06189169386555681, 'subsample': 0.741718379

In [104]:
study.best_trial.params

{'n_estimators': 200,
 'max_depth': 6,
 'learning_rate': 0.0874897674036755,
 'subsample': 0.9505081389818345,
 'colsample_bytree': 0.9724659902694157,
 'min_child_weight': 2,
 'reg_lambda': 10,
 'reg_alpha': 2}

In [78]:
# Configurar la URI de seguimiento de MLflow para que apunte al servidor local
mlflow.set_tracking_uri("http://localhost:5000")  # La dirección IP y puerto del servidor MLflow

In [94]:
# Esta función define una corrida del modelo, con entrenamiento y registro en MLflow
def run_mlflow(run_name="MLflow CE XGBoost-Best trial"):

    # Iniciamos una corrida de MLflow
    mlflow.start_run(run_name=run_name)
    run = mlflow.active_run()
    
    # MLflow asigna un ID al experimento y a la corrida
    experimentID = run.info.experiment_id
    runID = run.info.run_uuid

    # Entrenar el modelo con los mejores parametros
    clf =  XGBClassifier(**study.best_trial.params)
    clf.fit(X_train, y_train)
    # Evaluar el modelo
    y_pred = clf.predict(X_test)
    f1_score = metrics.f1_score(y_pred, y_test.values)

    # Registrar los parametros y métrica
    mlflow.log_params(study.best_trial.params)
    mlflow.log_metric("f1_score", study.best_trial.value)

    # Registrar el modelo
    mlflow.sklearn.log_model(clf, "xgboost-model")

    mlflow.end_run(status='FINISHED')
    return (experimentID, runID)

In [105]:
(experimentID, runID) = run_mlflow()
print("MLflow Run completed with run_id {} and experiment_id {}".format(runID, experimentID))

2024/11/09 15:58:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/09 15:58:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run MLflow CE XGBoost-Best trial at: http://localhost:5000/#/experiments/0/runs/2738e1afbb7c447ebdf47ecfd7e0ee92.
2024/11/09 15:58:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/0.


MLflow Run completed with run_id 2738e1afbb7c447ebdf47ecfd7e0ee92 and experiment_id 0


In [83]:
mlflow.end_run()

2024/11/09 13:30:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run MLflow CE XGBoost-Best trial at: http://localhost:5000/#/experiments/0/runs/e838516e6c834f4dae622102bf3b2f7f.
2024/11/09 13:30:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/0.


RestException: INVALID_PARAMETER_VALUE: The run e838516e6c834f4dae622102bf3b2f7f must be in 'active' lifecycle_stage.